# Pretrained model only

In [1]:
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.utils import shuffle

df = pd.read_csv('sample/Data_Entry_2017.csv')

diseases = ['Cardiomegaly','Emphysema','Effusion','Hernia','Nodule','Pneumothorax','Atelectasis','Pleural_Thickening','Mass','Edema','Consolidation','Infiltration','Fibrosis','Pneumonia']
#Number diseases
for disease in diseases :
    df[disease] = df['Finding Labels'].apply(lambda x: 1 if disease in x else 0)

# #test to perfect
# df = df.drop(df[df['Emphysema']==0][:-127].index.values)
    
#remove Y after age
df['Age']=df['Patient Age'].apply(lambda x: x[:-1]).astype(int)
df['Age Type']=df['Patient Age'].apply(lambda x: x[-1:])
df.loc[df['Age Type']=='M',['Age']] = df[df['Age Type']=='M']['Age'].apply(lambda x: round(x/12.)).astype(int)
df.loc[df['Age Type']=='D',['Age']] = df[df['Age Type']=='D']['Age'].apply(lambda x: round(x/365.)).astype(int)
# remove outliers
df = df.drop(df['Age'].sort_values(ascending=False).head(16).index)
df['Age'] = df['Age']/df['Age'].max()

#one hot data
# df = df.drop(df.index[4242])
df = df.join(pd.get_dummies(df['Patient Gender']))
df = df.join(pd.get_dummies(df['View Position']))

#random samples
df = shuffle(df)

#get other data
data = df[['Age', 'F', 'M', 'AP', 'PA']]
data = np.array(data)

labels = df[diseases].as_matrix()
files_list = ('sample/images/' + df['Image Index']).tolist()

# #test to perfect
# labelB = df['Emphysema'].tolist()

labelB = (df[diseases].sum(axis=1)>0).tolist()
labelB = np.array(labelB, dtype=int)

In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path, shape):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=shape)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)/255
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths, shape):
    list_of_tensors = [path_to_tensor(img_path, shape) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

train_labels = labelB[:89600][:, np.newaxis]
valid_labels = labelB[89600:100800][:, np.newaxis]
test_labels = labelB[100800:][:, np.newaxis]

train_data = data[:89600]
valid_data = data[89600:100800]
test_data = data[100800:]

img_shape = (64, 64)
train_tensors = paths_to_tensor(files_list[:89600], shape = img_shape)
valid_tensors = paths_to_tensor(files_list[89600:100800], shape = img_shape)
test_tensors = paths_to_tensor(files_list[100800:], shape = img_shape)

Using TensorFlow backend.
/home/aind2/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
100%|██████████| 11304/11304 [02:35<00:00, 72.47it/s]


### CNN model

In [4]:
import time

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator

# VGG16
# resnet50.ResNet50
# inception_v3.InceptionV3 299x299
# inception_resnet_v2.InceptionResNetV2 299x299

base_model = applications.VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=train_tensors.shape[1:])

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
# add_model.add(Conv2D(filters=512, 
#                  kernel_size=4, 
#                  strides=2, 
# #                  kernel_regularizer=regularizers.l2(0.01),
# #                  activity_regularizer=regularizers.l1(0.01),
#                  kernel_initializer=initializers.random_normal(stddev=0.01), 
#                  padding='same', 
#                  activation='relu', 
#                  input_shape=base_model.output_shape[1:]))
# # add_model.add(MaxPooling2D(pool_size=2))
# add_model.add(BatchNormalization())
# add_model.add(Flatten())
# add_model.add(Dropout(0.2))
# add_model.add(Dense(1024, activation='relu'))
add_model.add(Dropout(0.2))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.2))
add_model.add(Dense(50, activation='relu'))
add_model.add(Dropout(0.2))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.summary()
add_model.summary()

57040896/58889256 [============================>.] - ETA: 0s ETA: _________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_poo

In [5]:
from keras import backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [6]:
import keras.backend as K

model.compile(optimizer=optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True), 
              loss='binary_crossentropy', 
              metrics=['accuracy',
                      precision_threshold(threshold = 0.5), 
                       recall_threshold(threshold = 0.5), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.5)])

In [7]:
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import numpy as np

epochs = 20
batch_size = 32

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='auto')
log = CSVLogger('saved_models/log_pretrained_CNN.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/pretrainedVGG.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

start = time.time()

# model.fit(train_tensors, train_labels, 
#           validation_data=(valid_tensors, valid_labels),
#           epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, log, earlystop], verbose=1)

def train_generator(x, y, batch_size):
        train_datagen = ImageDataGenerator(
                        featurewise_center=False,  # set input mean to 0 over the dataset
                        samplewise_center=False,  # set each sample mean to 0
                        featurewise_std_normalization=False,  # divide inputs by std of the dataset
                        samplewise_std_normalization=False,  # divide each input by its std
                        zca_whitening=False,  # apply ZCA whitening
                        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                        horizontal_flip=True,  # randomly flip images
                        vertical_flip=False)  # randomly flip images
        generator = train_datagen.flow(x, y, batch_size=batch_size)
        while 1:
            x_batch, y_batch = generator.next()
            yield [x_batch, y_batch]

# Training with data augmentation. If shift_fraction=0., also no augmentation.
model.fit_generator(generator=train_generator(train_tensors, train_labels, batch_size),
                    steps_per_epoch=int(train_labels.shape[0] / batch_size),
                    validation_data=(valid_tensors, valid_labels),
                    epochs=epochs, callbacks=[checkpointer, log, earlystop], verbose=1)

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

Epoch 1/20
2800/2800 [==============================] - 384s - loss: 0.6433 - acc: 0.6360 - precision: 0.6261 - recall: 0.5302 - fbeta_score: 0.5911 - val_loss: 0.6085 - val_acc: 0.6794 - val_precision: 0.6966 - val_recall: 0.5468 - val_fbeta_score: 0.6519
Epoch 2/20
2800/2800 [==============================] - 386s - loss: 0.6203 - acc: 0.6693 - precision: 0.6620 - recall: 0.5899 - fbeta_score: 0.6364 - val_loss: 0.5993 - val_acc: 0.6864 - val_precision: 0.7023 - val_recall: 0.5623 - val_fbeta_score: 0.6609
Epoch 3/20
2800/2800 [==============================] - 386s - loss: 0.6141 - acc: 0.6753 - precision: 0.6658 - recall: 0.6047 - fbeta_score: 0.6439 - val_loss: 0.5969 - val_acc: 0.6945 - val_precision: 0.6972 - val_recall: 0.5999 - val_fbeta_score: 0.6688
Epoch 4/20
2800/2800 [==============================] - 384s - loss: 0.6106 - acc: 0.6790 - precision: 0.6668 - recall: 0.6136 - fbeta_score: 0.6480 - val_loss: 0.5982 - val_acc: 0.6871 - val_precision: 0.7096 - val_recall: 0.545

### Metric

In [8]:
model.load_weights('saved_models/pretrainedVGG.best.from_scratch.hdf5')
prediction = model.predict(test_tensors)

In [9]:
threshold = 0.5
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.675466 %
Recall: 0.619285 %
Fscore: 0.663429 %


In [10]:
K.eval(binary_accuracy(K.variable(value=test_labels),
                       K.variable(value=prediction)))

0.68763268

In [11]:
prediction[:30]

array([[ 0.86135578],
       [ 0.38395363],
       [ 0.49149001],
       [ 0.41301057],
       [ 0.69047374],
       [ 0.59208387],
       [ 0.47005421],
       [ 0.7447359 ],
       [ 0.76747203],
       [ 0.47276884],
       [ 0.65790629],
       [ 0.80368102],
       [ 0.55971575],
       [ 0.62390733],
       [ 0.79730672],
       [ 0.45695341],
       [ 0.2720097 ],
       [ 0.76029223],
       [ 0.39768952],
       [ 0.19952367],
       [ 0.84283119],
       [ 0.75410837],
       [ 0.5692758 ],
       [ 0.73594892],
       [ 0.39063299],
       [ 0.32207093],
       [ 0.4930805 ],
       [ 0.4202663 ],
       [ 0.47011665],
       [ 0.39033732]], dtype=float32)

# Training with extra data and spacial transformer

In [12]:
import time
import numpy as np

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.layers import Input, merge, concatenate
from spatial_transformer import SpatialTransformer

def locnet():
    b = np.zeros((2, 3), dtype='float32')
    b[0, 0] = 1
    b[1, 1] = 1
    W = np.zeros((64, 6), dtype='float32')
    weights = [W, b.flatten()]
    locnet = Sequential()

    locnet.add(Conv2D(16, (7, 7), padding='valid', input_shape=train_tensors.shape[1:]))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))
    locnet.add(Conv2D(32, (5, 5), padding='valid'))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))
    locnet.add(Conv2D(64, (3, 3), padding='valid'))
    locnet.add(MaxPooling2D(pool_size=(2, 2)))

    locnet.add(Flatten())
    locnet.add(Dense(128, activation='elu'))
    locnet.add(Dense(64, activation='elu'))
    locnet.add(Dense(6, weights=weights))

    return locnet

base_model = applications.VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=train_tensors.shape[1:])

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))

added0_model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

stn_model = Sequential()
stn_model.add(Lambda(
    lambda x: 2*x - 1.,
    input_shape=train_tensors.shape[1:],
    output_shape=train_tensors.shape[1:]))
stn_model.add(BatchNormalization())
stn_model.add(SpatialTransformer(localization_net=locnet(),
                                 output_size=train_tensors.shape[1:3]))

added_model = Model(inputs=stn_model.input, outputs=added0_model(stn_model.output))

inp = Input(batch_shape=(None, train_data.shape[1]))
# out = Dense(8)(inp)
extra_model = Model(input=inp, output=inp)

x = concatenate([added_model.output,
           extra_model.output])

# x = Dropout(0.5)(x)
# x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model =  Model(input=[added_model.input,
                extra_model.input],
                output=x)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1_input (InputLayer)      (None, 64, 64, 3)     0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 64, 64, 3)     0           lambda_1_input[0][0]             
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 64, 64, 3)     12          lambda_1[0][0]                   
____________________________________________________________________________________________________
spatial_transformer_1 (SpatialTr (None, 64, 64, 3)     247270      batch_normalization_1[0][0]      
___________________________________________________________________________________________

/home/aind2/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("in...)`
/home/aind2/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [13]:
from keras import backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [14]:
model.compile(optimizer=optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True), 
              loss='binary_crossentropy', 
              metrics=['accuracy',
                      precision_threshold(threshold = 0.5), 
                       recall_threshold(threshold = 0.5), 
                       fbeta_score_threshold(beta=0.5, threshold = 0.5)])

In [15]:
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

epochs = 20
batch_size = 32

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='auto')
log = CSVLogger('saved_models/log_pretrained_extradata_stn_CNN.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/log_pretrained_extradata_stn_CNN.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

start = time.time()

model.fit([train_tensors, train_data], train_labels, 
          validation_data=([valid_tensors, valid_data], valid_labels),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer, log, earlystop], verbose=1)

# def train_generator(x, y, batch_size):
#         train_datagen = ImageDataGenerator(
#                         featurewise_center=False,  # set input mean to 0 over the dataset
#                         samplewise_center=False,  # set each sample mean to 0
#                         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#                         samplewise_std_normalization=False,  # divide each input by its std
#                         zca_whitening=False,  # apply ZCA whitening
#                         rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#                         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#                         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#                         horizontal_flip=True,  # randomly flip images
#                         vertical_flip=False)  # randomly flip images
#         generator = train_datagen.flow(x, y, batch_size=batch_size)
#         while 1:
#             x_batch, y_batch = generator.next()
#             yield [x_batch, y_batch]

# # Training with data augmentation. If shift_fraction=0., also no augmentation.
# model.fit_generator(generator=train_generator(train_tensors, train_labels, batch_size),
#                     steps_per_epoch=int(train_labels.shape[0] / batch_size),
#                     validation_data=(valid_tensors, valid_labels),
#                     epochs=epochs, callbacks=[checkpointer, log, earlystop], verbose=1)

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

/home/aind2/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 89600 samples, validate on 11200 samples
Epoch 1/20
89600/89600 [==============================] - 431s - loss: 0.6440 - acc: 0.6434 - precision: 0.6296 - recall: 0.5685 - fbeta_score: 0.6068 - val_loss: 0.6118 - val_acc: 0.6751 - val_precision: 0.6260 - val_recall: 0.7379 - val_fbeta_score: 0.6418
Epoch 2/20
89600/89600 [==============================] - 433s - loss: 0.6160 - acc: 0.6760 - precision: 0.6647 - recall: 0.6083 - fbeta_score: 0.6441 - val_loss: 0.5953 - val_acc: 0.6919 - val_precision: 0.6656 - val_recall: 0.6749 - val_fbeta_score: 0.6624
Epoch 3/20
89600/89600 [==============================] - 433s - loss: 0.6099 - acc: 0.6819 - precision: 0.6704 - recall: 0.6165 - fbeta_score: 0.6511 - val_loss: 0.5941 - val_acc: 0.6946 - val_precision: 0.6575 - val_recall: 0.7096 - val_fbeta_score: 0.6629
Epoch 4/20
89600/89600 [==============================] - 432s - loss: 0.6062 - acc: 0.6858 - precision: 0.6746 - recall: 0.6246 - fbeta_score: 0.6564 - val_loss: 0.5909 - v

In [16]:
model.load_weights('saved_models/log_pretrained_extradata_stn_CNN.best.from_scratch.hdf5')
prediction = model.predict([test_tensors, test_data])

In [17]:
threshold = 0.5
beta = 0.5

pre = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
rec = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))
fsc = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),
                                   K.variable(value=prediction)))

print ("Precision: %f %%\nRecall: %f %%\nFscore: %f %%"% (pre, rec, fsc))

Precision: 0.684010 %
Recall: 0.621206 %
Fscore: 0.670454 %


In [18]:
K.eval(binary_accuracy(K.variable(value=test_labels),
                       K.variable(value=prediction)))

0.69338286